# Fitting 2D images within gammapy

Gammapy does not have any special handling for 2D images, but treats them as a subset of maps. Thus, classical 2D image analysis can be done in 2 independent ways:- 
1. Using the sherpa pacakge, see: https://github.com/gammapy/gammapy/blob/master/tutorials/image_fitting_with_sherpa.ipynb\n",
2. Within gammapy, by assuming 2D analysis to be a sub-set of the generalised `maps`. Thus, analysis should proceed exactly as demonstrated in https://github.com/gammapy/gammapy/blob/master/tutorials/analysis_3d.ipynb, taking care of a few things that we mention in this tutorial
We consider 2D `images` to be a special case of 3D `maps`, ie, maps with only one energy bin. This is a major difference while analysing in `sherpa`, where the `maps` must not contain any energy axis.\n",
In this tutorial, we do a classical image analysis using three example observations of the Galactic center region with CTA - ie, study the source flux and morphology.



## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.extern.pathlib import Path
from gammapy.data import DataStore
from gammapy.irf import EnergyDispersion, make_mean_psf, make_mean_edisp
from gammapy.maps import WcsGeom, MapAxis, Map, WcsNDMap
from gammapy.cube import MapMaker, MapEvaluator, PSFKernel, MapFit
from gammapy.cube.models import SkyModel
from gammapy.spectrum.models import PowerLaw
from gammapy.image.models import SkyGaussian, SkyPointSource
from regions import CircleSkyRegion


## Prepare modeling input data

### The counts, exposure and the background maps
This is the same drill - use `DataStore` object to access the CTA observations and retrieve a list of observations by passing the observations IDs to the `.get_observations()` method, then use `MapMaker` to make the maps.

In [ ]:
# Define which data to use and print some information
data_store = DataStore.from_dir("$GAMMAPY_DATA/cta-1dc/index/gps/")
data_store.info()
print(
    "Total observation time (hours): ",
    data_store.obs_table["ONTIME"].sum() / 3600,
)
print("Observation table: ", data_store.obs_table.colnames)

In [ ]:
# Select some observations from these dataset by hand
obs_ids = [110380, 111140, 111159]
observations = data_store.get_observations(obs_ids)

In [ ]:
energy_axis = MapAxis.from_edges(
    np.logspace(-1.0, 1.0, 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(10, 8),
    coordsys="GAL",
    proj="CAR",
    axes=[energy_axis],
)

Note that even when doing a 2D analysis, it is better to use fine energy bins in the beginning and then sum them over. This is to ensure that the background shape can be approximated by a power law function in each energy bin.

In [ ]:
%%time
maker = MapMaker(geom, offset_max=4.0 * u.deg)
maps = maker.run(observations)

In [ ]:
maps

As we can see, the maps now have multiple bins in energy. We need to squash them to have one bin, and this can be done by specifying `keep_dims = True` while calling `make_images()`

In [ ]:
maps2D = maker.make_images(keepdims=True)

For a typical 2D analysis, using an energy dispersion usually does not make sense. A PSF map can be made as in the regular 3D case.

In [ ]:
# mean PSF
geom2d = maps2D['exposure'].geom
src_pos = SkyCoord(0, 0, unit="deg", frame="galactic")
table_psf = make_mean_psf(observations, src_pos)

# PSF kernel used for the model convolution
psf_kernel = PSFKernel.from_table_psf(table_psf, geom2d, max_radius="0.3 deg")

Now, the analysis proceeds as usual. Just take care to use the proper geometry in this case.

## Define a mask

In [ ]:
mask = Map.from_geom(geom2d)

region = CircleSkyRegion(center=src_pos, radius=0.6 * u.deg)
mask.data = mask.geom.region_mask([region])

## Modelling the source

This is the important thing to note in this analysis. Since modelling and fitting in `gammapy.maps` needs to have a combination of spectral models, we have to use a dummy Powerlaw as for the specrtal model and fix its index to 2

In [ ]:
spatial_model = SkyPointSource(lon_0="0.01 deg", lat_0="0.01 deg")
spectral_model = PowerLaw(
    index=2.0, amplitude="3e-12 cm-2 s-1 TeV-1", reference="1 TeV"
)
model = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)
model.parameters["index"].frozen = True

In [ ]:
fit = MapFit(
    model=model,
    counts=maps2D["counts"],
    exposure=maps2D["exposure"],
    background=maps2D["background"],
    mask=mask,
    psf=psf_kernel,
)

In [ ]:
%%time
result = fit.run(optimize_opts={"print_level": 1})

The output of the fit does not look very meaningful. To see the actual best-fit parameters, do a print on the result

In [ ]:
print(result.model)

Again, the differential flux does not convey much information. The important parameters during a 2D fit are the integral flux and the spatial parameters. To get the total flux, we compute the integral flux of the power law model.

In [ ]:
integral_flux = model.spectral_model.integral(energy_axis.edges[0], energy_axis.edges[-1])
integral_flux

## Todo: Demonstrate plotting a flux map

## Exercises
1. Plot residual maps as done in the `analysis_3d` notebook
2. Iteratively add and fit sources as explained in `image_fitting_with_sherpa` notebook
